In [1]:
import glob

import polars as pl

In [3]:
DATA = './data/nyc-taxi/*.parquet'
files = list(glob.glob(DATA))

In [6]:
%%time

# this method avoids a PanicException: this operation is not implemented/valid for this dtype: Null
df = pl.concat(
    [
        pl.read_parquet(file).drop('airport_fee').lazy()
        for file in files
    ]
)

CPU times: user 17.4 s, sys: 4.09 s, total: 21.5 s
Wall time: 11 s


In [7]:
df.collect().shape

(91809949, 18)

In [8]:
df.collect().estimated_size('gb')

12.437642632983625

In [9]:
df.collect().head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
i64,datetime[ns],datetime[ns],f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64
1,2022-10-01 00:03:41,2022-10-01 00:18:39,1.0,1.7,1.0,"""N""",249,107,1,9.5,3.0,0.5,2.65,0.0,0.3,15.95,2.5
2,2022-10-01 00:14:30,2022-10-01 00:19:48,2.0,0.72,1.0,"""N""",151,238,2,5.5,0.5,0.5,0.0,0.0,0.3,9.3,2.5
2,2022-10-01 00:27:13,2022-10-01 00:37:41,1.0,1.74,1.0,"""N""",238,166,1,9.0,0.5,0.5,2.06,0.0,0.3,12.36,0.0
1,2022-10-01 00:32:53,2022-10-01 00:38:55,0.0,1.3,1.0,"""N""",142,239,1,6.5,3.0,0.5,2.05,0.0,0.3,12.35,2.5
1,2022-10-01 00:44:55,2022-10-01 00:50:21,0.0,1.0,1.0,"""N""",238,166,1,6.0,0.5,0.5,1.8,0.0,0.3,9.1,0.0


In [10]:
%%time

(
    df.with_columns(
        is_congested=pl.col('congestion_surcharge').fill_null(0) > 0
    )
    .groupby('DOLocationID')
    .agg(
        mean_fare=pl.mean('fare_amount'),
        count=pl.count(),
        congestion=pl.mean('is_congested')
    )
    .sort('count', descending=True)
    .collect()
)

CPU times: user 4.51 s, sys: 3.63 s, total: 8.14 s
Wall time: 17.2 s


DOLocationID,mean_fare,count,congestion
i64,f64,u32,f64
236,9.600091,4109242,0.966388
237,9.118917,3845516,0.975696
161,10.634796,3165651,0.971493
170,10.632019,2730697,0.967348
141,9.888821,2567317,0.970547
239,10.588303,2554702,0.970024
142,10.338966,2513474,0.970639
48,12.053984,2399829,0.964962
162,10.76548,2394659,0.972309
